In [58]:
reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [310]:
import bs4, re
import numpy as np
import pandas as pd
from pandas import DataFrame
from collections import Counter
import plotly.plotly as py
import cufflinks

py.sign_in('1IfByLAN2IfByC','r50yawfp79')


In [68]:
### LOAD IN THE HTML FILE TO A LIST

filename = '/Users/michael/Desktop/Netflix.html'
with open(filename) as f:
    HTML = f.read()

In [169]:
parsedHTML = bs4.BeautifulSoup(HTML)


In [306]:
def netflixParser(HTMLstring):
    parsedHTML = bs4.BeautifulSoup(HTML)

    names = parsedHTML.findAll('div', attrs={'class':'col title'})
    dates = parsedHTML.findAll('div', attrs={'class':'col date nowrap'})
    
    if len(names) != len(dates):
        assert len(names) == len(dates)
    
    ## construct a dataframe
    netflix = np.zeros((len(names), 3))
    netflix = DataFrame(netflix, columns=['Date','Series Name', 'Episode Name'])
    
    
    ## loop through the name/date list and extract
    for i in range(len(names)):
        try:
            name = names[i].find('span', attrs={'class': 'seriestitle'})
            netflix['Series Name'].iloc[i] = name.text   
            netflix['Date'].loc[i] = dates[i].text
            netflix['Episode Name'].iloc[i] = re.findall(r'\</span>:(.*?)\</a>', str(names[i]))[0]
        except AttributeError:
            pass
        

    ## clean up the zeros
    netflix = netflix[netflix.Date != 0]
    ## make new dataframe 
    showCounts = Counter(netflix['Series Name'])
    showCounts = netflix.from_dict(showCounts, orient='index')
    showCounts.columns = [ 'Count']
    showCounts = showCounts.sort('Count', ascending=False)
    return netflix, showCounts

    

In [307]:
netflix, showCounts = netflixParser(HTML)

In [317]:
showCounts.iplot(kind='scatter', mode='markers', y='Count', world_readable=True)